In [33]:
using Finch
using SparseArrays
using Metatheory
using Metatheory.EGraphs
using TermInterface
using PrettyPrinting
include("../Source/Spartan.jl")
include("../Source/UtilityFuncs.jl")

uniform_fiber (generic function with 1 method)

In [34]:
is = ["i", "l"]
A = InputTensor("A", ["i", "j"], uniform_fiber([10000, 100], .01))
B = InputTensor("B", ["j", "l"], uniform_fiber([100, 10000], .01))
C = InputTensor("C", ["j", "l"], uniform_fiber([100, 10000], .01))
output_order = ["j", "i"]

verbose = 1
# Sum_i Sum_l B[j,l]*A[i,j]*A[i,j]
D = A * B * C
E = Reorder(ReduceDim(+, ["l"],  D), ["j","i"])
:(MapJoin($*, $A, MapJoin($*, $B, $C)))

output_tensor_1 = spartan(E, verbose=verbose, optimize=false)
#println(output_tensor)

global_index_order = ["i", "j", "l"]
# (Sum_l B[j,l])*(Sum_i A[i,j] * A[i,j])
output_tensor_2 = spartan(E, verbose=verbose, global_index_order = global_index_order, optimize=true)

global_index_order = ["l", "j", "i"]
output_tensor_3 = spartan(E, verbose=verbose, global_index_order = global_index_order, optimize=true)

global_index_order = ["j", "l", "i"]
output_tensor_4 = spartan(E, verbose=verbose, global_index_order = global_index_order, optimize=true)


global_index_order = ["i", "l", "j"]
output_tensor_5 = spartan(E, verbose=verbose, global_index_order = global_index_order, optimize=true)

println("Is Correct?: ", output_tensor_1 == output_tensor_2 && output_tensor_2 == output_tensor_3 && 
                                output_tensor_3 == output_tensor_4 && output_tensor_4 == output_tensor_5)

Expression: Reorder(
   ReduceDim(+,["l"],
      MapJoin(*,
         MapJoin(*,
            InputTensor(A,["i", "j"],FIBER,Any["i", "j", "l"]),
            InputTensor(B,["j", "l"],FIBER,Any["i", "j", "l"])),
         InputTensor(C,["j", "l"],FIBER,Any["i", "j", "l"]))),["j", "i"])
Time to Execute: 

0.47436554


Optimized Expression: Reorder(
   ReduceDim(+,["l"],
      MapJoin(*,
         MapJoin(*,
            InputTensor(B,["j", "l"],FIBER,["i", "j", "l"]),
            InputTensor(C,["j", "l"],FIBER,["i", "j", "l"])),
         InputTensor(A,["i", "j"],FIBER,["i", "j", "l"]))),["j", "i"])
Time to Execute: 

0.098742641
Optimized Expression: ReduceDim(+,["l"],
   MapJoin(*,
      MapJoin(*,
         Reorder(
            InputTensor(B,["j", "l"],FIBER,["l", "j", "i"]),["l", "j", "i"]),
         Reorder(
            InputTensor(C,["j", "l"],FIBER,["l", "j", "i"]),["l", "j", "i"])),
      Reorder(
         InputTensor(A,["i", "j"],FIBER,["l", "j", "i"]),["l", "j", "i"])))
Time to Execute: 

0.152786531
Optimized Expression: ReduceDim(+,["l"],
   MapJoin(*,
      MapJoin(*,
         InputTensor(B,["j", "l"],FIBER,["j", "l", "i"]),
         InputTensor(C,["j", "l"],FIBER,["j", "l", "i"])),
      Reorder(
         InputTensor(A,["i", "j"],FIBER,["j", "l", "i"]),["j", "l", "i"])))
Time to Execute: 0.080366201
Optimized Expression: Reorder(
   ReduceDim(+,["l"],
      MapJoin(*,
         MapJoin(*,
            Reorder(
               InputTensor(B,["j", "l"],FIBER,["i", "l", "j"]),["i", "l", "j"]),
            Reorder(
               InputTensor(C,["j", "l"],FIBER,["i", "l", "j"]),["i", "l", "j"])),
         InputTensor(A,["i", "j"],FIBER,["i", "l", "j"]))),["j", "i"])
Time to Execute: 

0.075239145
Is Correct?: true


In [35]:
is = ["i", "j"]
B = InputTensor("B", ["i", "k", "l"], uniform_fiber([1000, 1000, 1000], .00001))
D = InputTensor("D", ["l", "j"], uniform_fiber([1000, 1000], 0.1))
C = InputTensor("C", ["k", "j"], uniform_fiber([1000, 1000], 0.1))

verbose = 3
MTTKRP = ReduceDim(+, is, MapJoin(*, B, MapJoin(*, C, D)))

output_tensor_1 = spartan(MTTKRP, optimize=false, verbose=verbose)

output_tensor_2 = spartan(MTTKRP, optimize=true, verbose=verbose)

MTTKRP_hand_opt = ReduceDim(+, ["i"], MapJoin(*, B, ReduceDim(+, ["j"], MapJoin(*, C, D))))
output_tensor_2 = spartan(MTTKRP_hand_opt, optimize=false, verbose=verbose)



Expression: ReduceDim(+,["i", "j"],
   MapJoin(*,
      InputTensor(B,["i", "k", "l"],FIBER,Any["i", "j", "k", "l"]),
      MapJoin(*,
         Reorder(
            InputTensor(C,["k", "j"],FIBER,Any["i", "j", "k", "l"]),Any["i", "j", "k", "l"]),
         Reorder(
            InputTensor(D,["l", "j"],FIBER,Any["i", "j", "k", "l"]),Any["i", "j", "k", "l"]))))
1


2
3
4
4
Kernel: 

ReorderExpr(["i", "j", "k", "l"], InputExpr

("A", ["l", "j"], AccessProtocol[t_walk, t_walk], TensorStats(["l", "j"], Dict

("j" => 1000, "l" => 1000), 100362.0, 0, Any["i", "j", "k", "l"])))
Output Order: ["j", "l"]
Loop Order: ["l", "j"]
Expected Output Tensor Size: 

...

Expected Output Tensor Size: 999957.1674487703
Output Tensor Size: 999960
2


Kernel: OperatorExpr(*, InputExpr[InputExpr("A", ["i", "k", "l"], AccessProtocol[t_walk, t_walk, t_walk], TensorStats(["i", "k", "l"], Dict("k" => 1000, "l" => 1000, "i" => 1000), 9974.0, 0, Any["i", "j", "k", "l"])), InputExpr("B", ["k", "l"], AccessProtocol[t_walk, t_walk], TensorStats(["k", "l"], Dict("k" => 1000, "l" => 1000), 999957.1674487703, 0, ["i", "j", "k", "l"]))])
Output Order: ["i", "k", "l"]
Loop Order: ["l", "k", "i"]


Expected Output Tensor Size: 9973.572788134034
Output Tensor Size: 9974
1
Kernel: AggregateExpr(+, ["i"], InputExpr("A", ["i", "k", "l"], AccessProtocol[t_walk, t_walk, t_walk], TensorStats(["i", "k", "l"], Dict("k" => 1000, "l" => 1000, "i" => 1000), 9973.572788134034, 0, Any["i", "j", "k", "l"])))
Output Order: ["k", "l"]
Loop Order: ["l", "k", "i"]


Expected Output Tensor Size: 9924.050891170322
Output Tensor Size: 9933
Time to Execute: 6.513719902


SparseHash (0) [:,1:1000]
├─[:,1]: SparseHash (0) [1:1000]
│ ├─[120]: 10
│ ├─[195]: 8
│ │ ⋮
│ ├─[799]: 13
│ ├─[810]: 12
├─[:,2]: SparseHash (0) [1:1000]
│ ├─[47]: 13
│ ├─[164]: 9
│ │ ⋮
│ ├─[934]: 9
│ ├─[948]: 11
│ ⋮
├─[:,999]: SparseHash (0) [1:1000]
│ ├─[104]: 13
│ ├─[122]: 5
│ │ ⋮
│ ├─[715]: 11
│ ├─[950]: 15
├─[:,1000]: SparseHash (0) [1:1000]
│ ├─[46]: 12
│ ├─[643]: 9
│ ├─[779]: 10
│ ├─[786]: 7
│ ├─[927]: 9

In [36]:
is = ["i", "j"]
X = InputTensor("X", ["i", "j"], uniform_fiber([100, 100], .5))
u = InputTensor("u", ["i"], uniform_fiber([100], 1.0))
v = InputTensor("v", ["j"], uniform_fiber([100], 1.0))

verbose = 1
# Sum_i Sum_j (X[i,j]+u[i]*v[j])
output_tensor_1 = spartan(ReduceDim(+, is, MapJoin(^, MapJoin(+, X, MapJoin(*, u, v)), 2)), verbose=verbose, optimize=false)

# (Sum_i u[i]^2)*(Sum_j v[j]^2) + (Sum_i u[i] * (Sum_j X[i,j] * V[j])) + (Sum_i u[i] * (Sum_j X[i,j] * V[j])) + (Sum_[i,j] X[i,j]^2)
output_tensor_2 = spartan(ReduceDim(+, is, MapJoin(^, MapJoin(+, X, MapJoin(*, u, v)), 2)), verbose=verbose, optimize=true)

println("Is Correct?: ", output_tensor_1 == output_tensor_2)

Expression: ReduceDim(+,["i", "j"],
   MapJoin(^,
      MapJoin(+,
         InputTensor(X,["i", "j"],FIBER,Any["i", "j"]),
         MapJoin(*,
            InputTensor(u,["i"],FIBER,Any["i", "j"]),
            InputTensor(v,["j"],FIBER,Any["i", "j"]))),2))
Time to Execute: 

2.489146799


Optimized Expression: MapJoin(+,
   MapJoin(+,
      ReduceDim(+,["i"],
         MapJoin(*,
            InputTensor(u,["i"],FIBER,["i", "j"]),
            ReduceDim(+,["j"],
               MapJoin(*,
                  InputTensor(X,["i", "j"],FIBER,["i", "j"]),
                  InputTensor(v,["j"],FIBER,["i", "j"]))))),
      MapJoin(+,
         MapJoin(*,
            ReduceDim(+,["j"],
               MapJoin(^,
                  InputTensor(v,["j"],FIBER,["i", "j"]),2)),
            ReduceDim(+,["i"],
               MapJoin(^,
                  InputTensor(u,["i"],FIBER,["i", "j"]),2))),
         ReduceDim(+,["i"],
            MapJoin(*,
               InputTensor(u,["i"],FIBER,["i", "j"]),
               ReduceDim(+,["j"],
                  MapJoin(*,
                     InputTensor(X,["i", "j"],FIBER,["i", "j"]),
                     InputTensor(v,["j"],FIBER,["i", "j"]))))))),
   ReduceDim(+,["j", "i"],
      MapJoin(^,
         InputTensor(X,["i", "j"],FIBER,["i", "j"]),2)))
Tim

2.38932886
Is Correct?: true


In [1]:
is = ["i", "j", "k"]
E_fiber = uniform_fiber([1000, 1000], .04)
E_1 = InputTensor("E", ["i", "j"], E_fiber)
E_2 = InputTensor("E", ["j", "k"], E_fiber)
E_3 = InputTensor("E", ["k", "l"], E_fiber)
E_4 = InputTensor("E", ["l", "i"], E_fiber)

verbose = 1
D = E_1 * E_2 * E_3 * E_4
query = ReduceDim(+, is, D)
output_1 = spartan(query, verbose=verbose, optimize = false)
output_2 = spartan(query, verbose=verbose, optimize = true)

UndefVarError: UndefVarError: `uniform_fiber` not defined

In [2]:
# Suppose you have a network of power nodes and lines. Further, you need to take weather events/outages into account.
# The following query provides the total available power at each node.
num_nodes = 1000
height = 100
width = 100

# Because we're dealing with positive numbers, the following is true.
Finch.isassociative(::Finch.DefaultAlgebra, ::typeof(min)) = true
Finch.iscommutative(::Finch.DefaultAlgebra, ::typeof(min)) = true
Finch.isannihilator(::Finch.DefaultAlgebra, ::typeof(min), x) = x <= 0

transmission_capacity = InputTensor("transmission_capacity", ["i", "j"], uniform_fiber([num_nodes, num_nodes], 5.0/num_nodes))

node_location_data =  uniform_fiber([num_nodes, height, width], 1.0/height/width)
node_location = InputTensor("node_location", ["i", "lat", "long"], node_location_data)
neighbor_node_location = InputTensor("neighbor_node_location", ["j", "lat2", "long2"], node_location_data)

weather_events_data = uniform_fiber([height, width], .01, default_value = 1, non_default_value = 0)
weather_events = InputTensor("weather_events", ["lat", "long"], weather_events_data)
neighbor_weather_events = InputTensor("neighbor_weather_events", ["lat2", "long2"], weather_events_data)

outages_data = uniform_fiber([num_nodes], .1, default_value = 1, non_default_value = 0)
node_outage_percents = InputTensor("node_outage_percents", ["i"], outages_data)
neighbor_node_outage_percents = InputTensor("neighbor_node_outage_percents", ["j"], outages_data)

node_power_data = uniform_fiber([num_nodes], 1.0)
node_power = InputTensor("node_power", ["i"], node_power_data)
neighbor_node_power = InputTensor("neighbor_node_power", ["j"], node_power_data)

available_power_query = ReduceDim(+, ["lat", "long"],node_location * weather_events * node_outage_percents * node_power) + 
                        ReduceDim(+, ["j"], min(ReduceDim(+, ["lat2", "long2"], neighbor_weather_events* neighbor_node_outage_percents 
                                                                        * neighbor_node_location * neighbor_node_power),
                                                             transmission_capacity))

verbose = 1
total_neighbor_power_1 = spartan(available_power_query, verbose=verbose, optimize=true)

total_neighbor_power_2 = spartan(available_power_query, verbose=verbose, optimize=false)

println("Is Equivalent: ", total_neighbor_power_1 == total_neighbor_power_2)

UndefVarError: UndefVarError: `Finch` not defined